In [1]:
# Import packages
import os
import random
from PIL import Image
from torch import nn
import torch 
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import json 
from src.config import PROCESSED_DATA_DIR, RAW_DATA_DIR
import torch.optim as optim

from model_architectures.v01 import HHP01
from model_architectures.v02 import HHP02

from utils import train_validate_model, modify_model_output, test_model

image_path = PROCESSED_DATA_DIR / "pizza_hamburger_hotdog_20_percent"
train_dir = image_path / 'train' 
test_dir = image_path / 'test'
valid_dir = image_path / 'valid'

device ='cuda' if torch.cuda.is_available() else 'cpu'

2024-06-23 12:32:09.601 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Git\hamburger-hotdog-pizza-classifier


In [2]:
config_path = 'params.json'

with open(config_path, 'r') as config_file:
   config = json.load(config_file)
#print(json.dumps(config, indent=2))

In [3]:
# Load hyperparams from params.json

learning_rate = config['model_params']['learning_rate']
batch_size = config['model_params']['batch_size']
num_epochs = config['model_params']['num_epochs']
dropout_rate = config['model_params']['dropout_rate']
optimizer = config['model_params']['optimizer']
loss_function = config['model_params']['loss_function']
metrics = config['model_params']['metrics']
output_shape = config['model_params']['output_shape']

In [4]:
# Prepare data into dataloader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
])

train_data = ImageFolder(train_dir, transform=transform)
valid_data = ImageFolder(valid_dir, transform=transform)
test_data = ImageFolder(test_dir, transform=transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Start with Pre-Trained Models

Let's choose some popular CNN architectures as a starting point

First, we need to load these pretrained models

In [6]:
num_classes = 3
alexnet = modify_model_output('alexnet', num_classes, device)
vgg16 = modify_model_output('vgg16', num_classes, device)
resnet50 = modify_model_output('resnet50', num_classes, device)

## 1. Trying ResNet50

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.3)
train_validate_model(num_epochs=10, model=resnet50, train_loader=train_loader, valid_loader=valid_loader, criterion=criterion, optimizer=optimizer, device=device)

Epoch 1, Train Loss: 1.1049, Train Accuracy: 37.41%
Epoch 1, Validation Loss: 1.0913, Validation Accuracy: 41.67%
Epoch 2, Train Loss: 1.0940, Train Accuracy: 39.63%
Epoch 2, Validation Loss: 1.0859, Validation Accuracy: 46.11%
Epoch 3, Train Loss: 1.0900, Train Accuracy: 39.63%
Epoch 3, Validation Loss: 1.0778, Validation Accuracy: 44.44%
Epoch 4, Train Loss: 1.0750, Train Accuracy: 42.59%
Epoch 4, Validation Loss: 1.0704, Validation Accuracy: 44.44%
Epoch 5, Train Loss: 1.0864, Train Accuracy: 44.44%
Epoch 5, Validation Loss: 1.0614, Validation Accuracy: 48.33%
Epoch 6, Train Loss: 1.0653, Train Accuracy: 50.37%
Epoch 6, Validation Loss: 1.0568, Validation Accuracy: 50.56%
Epoch 7, Train Loss: 1.0602, Train Accuracy: 50.37%
Epoch 7, Validation Loss: 1.0493, Validation Accuracy: 52.78%
Epoch 8, Train Loss: 1.0490, Train Accuracy: 55.19%
Epoch 8, Validation Loss: 1.0448, Validation Accuracy: 51.67%
Epoch 9, Train Loss: 1.0483, Train Accuracy: 57.04%
Epoch 9, Validation Loss: 1.0375, Va

## 2. Trying VGG16

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.3)
train_validate_model(num_epochs=20, model=vgg16, train_loader=train_loader, valid_loader=valid_loader, criterion=criterion, optimizer=optimizer, device=device)

Epoch 1, Train Loss: 0.3330, Train Accuracy: 87.78%
Epoch 1, Validation Loss: 0.3984, Validation Accuracy: 85.56%
Epoch 2, Train Loss: 0.3374, Train Accuracy: 89.63%
Epoch 2, Validation Loss: 0.3805, Validation Accuracy: 86.67%
Epoch 3, Train Loss: 0.3017, Train Accuracy: 87.04%
Epoch 3, Validation Loss: 0.3608, Validation Accuracy: 86.67%
Epoch 4, Train Loss: 0.2949, Train Accuracy: 92.96%
Epoch 4, Validation Loss: 0.3505, Validation Accuracy: 87.78%
Epoch 5, Train Loss: 0.2362, Train Accuracy: 90.74%
Epoch 5, Validation Loss: 0.3333, Validation Accuracy: 88.33%
Epoch 6, Train Loss: 0.2540, Train Accuracy: 91.85%
Epoch 6, Validation Loss: 0.3371, Validation Accuracy: 88.89%
Epoch 7, Train Loss: 0.2595, Train Accuracy: 89.63%
Epoch 7, Validation Loss: 0.3384, Validation Accuracy: 87.78%
Epoch 8, Train Loss: 0.1867, Train Accuracy: 92.96%
Epoch 8, Validation Loss: 0.3120, Validation Accuracy: 88.89%
Epoch 9, Train Loss: 0.1916, Train Accuracy: 92.59%
Epoch 9, Validation Loss: 0.3074, Va

## 3. Trying AlexNet

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.3)
train_validate_model(num_epochs=10, model=alexnet, train_loader=train_loader, valid_loader=valid_loader, criterion=criterion, optimizer=optimizer, device=device)

Epoch 1, Train Loss: 1.1237, Train Accuracy: 41.48%
Epoch 1, Validation Loss: 1.0073, Validation Accuracy: 50.00%
Epoch 2, Train Loss: 0.9357, Train Accuracy: 56.67%
Epoch 2, Validation Loss: 0.8322, Validation Accuracy: 69.44%
Epoch 3, Train Loss: 0.7515, Train Accuracy: 70.74%
Epoch 3, Validation Loss: 0.7442, Validation Accuracy: 71.11%
Epoch 4, Train Loss: 0.6730, Train Accuracy: 74.44%
Epoch 4, Validation Loss: 0.6634, Validation Accuracy: 76.67%
Epoch 5, Train Loss: 0.5943, Train Accuracy: 78.89%
Epoch 5, Validation Loss: 0.5952, Validation Accuracy: 76.67%
Epoch 6, Train Loss: 0.5595, Train Accuracy: 75.56%
Epoch 6, Validation Loss: 0.6045, Validation Accuracy: 77.78%
Epoch 7, Train Loss: 0.5328, Train Accuracy: 76.67%
Epoch 7, Validation Loss: 0.5419, Validation Accuracy: 81.11%
Epoch 8, Train Loss: 0.4717, Train Accuracy: 83.33%
Epoch 8, Validation Loss: 0.5859, Validation Accuracy: 73.89%
Epoch 9, Train Loss: 0.4556, Train Accuracy: 81.85%
Epoch 9, Validation Loss: 0.5568, Va

In [19]:
import torchvision.models as models
vgg11 = models.vgg11(weights='DEFAULT')
num_features = vgg11.classifier[6].in_features
vgg11.classifier[6] = nn.Linear(num_features, 3)

# Training/Validation Loop
train_validate_model(num_epochs=10, train_loader=train_loader, valid_loader=valid_loader, model=model, criterion=criterion, optimizer=optimizer, device=device)

Epoch 1, Train Loss: 1.0988, Train Accuracy: 33.70%
Epoch 1, Validation Loss: 1.0999, Validation Accuracy: 30.56%
Epoch 2, Train Loss: 1.0982, Train Accuracy: 33.70%
Epoch 2, Validation Loss: 1.1000, Validation Accuracy: 30.56%
Epoch 3, Train Loss: 1.0982, Train Accuracy: 33.70%
Epoch 3, Validation Loss: 1.0998, Validation Accuracy: 30.56%
Epoch 4, Train Loss: 1.0996, Train Accuracy: 33.70%
Epoch 4, Validation Loss: 1.0999, Validation Accuracy: 30.56%
Epoch 5, Train Loss: 1.0985, Train Accuracy: 33.70%
Epoch 5, Validation Loss: 1.0997, Validation Accuracy: 30.56%
Epoch 6, Train Loss: 1.0983, Train Accuracy: 33.70%
Epoch 6, Validation Loss: 1.0997, Validation Accuracy: 30.56%
Epoch 7, Train Loss: 1.0987, Train Accuracy: 33.70%
Epoch 7, Validation Loss: 1.0999, Validation Accuracy: 30.56%


KeyboardInterrupt: 

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.3)

train_validate_model(num_epochs=10, train_loader=train_loader, valid_loader=valid_loader, model=model, criterion=criterion, optimizer=optimizer, device=device)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import vgg16, VGG16_Weights
from torchvision.models import alexnet, AlexNet_Weights
from torchvision.models import incep

In [24]:
#model = resnet50(weights=ResNet50_Weights.DEFAULT).to(device)
backbone = model.features
#backbone[-1].out_channels
backbone

AttributeError: 'ResNet' object has no attribute 'features'

In [28]:
num_classes = 3
alexnet = modify_model_output('alexnet', num_classes)
vgg16 = modify_model_output('vgg16', num_classes)
resnet50 = modify_model_output('resnet50', num_classes)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\MLDev/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:07<00:00, 32.5MB/s] 
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\MLDev/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:22<00:00, 25.0MB/s] 
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\MLDev/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:03<00:00, 33.5MB/s] 

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights
model = resnet50(weights=ResNet50_Weights.DEFAULT).to(device)
train_validate_model(num_epochs=100, train_loader=train_loader, valid_loader=valid_loader, model=model.softmax(3), criterion=criterion, optimizer=optimizer, device=device)
#num_features = resnet50.classifier[6].in_features
#vgg11.classifier[6] = nn.Linear(num_features, 3)

In [26]:
model = models.vgg16(weights='DEFAULT')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\MLDev/.cache\torch\hub\checkpoints\vgg16-397923af.pth
 27%|██▋       | 144M/528M [00:05<00:14, 27.7MB/s] 


KeyboardInterrupt: 